<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
#!pip install sqlalchemy==1.3.9
#!pip install -q pandas==1.1.5

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

In [5]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [6]:
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


___

## Preprocessing
After having completed most of the lab using the base code that was provided to get started with the database, I quickly noticed some insconsistencies in the formatting of columns and some of the values in the database. I decided to add some cleaning steps in the notebook to ensure consistency in the formatting and values within the columns. 

In [7]:
# In database design and development, it's a best practice to have a consistent naming convention for columns. 
# PAYLOAD_MASS__KG_ is considered an irregular convention due to the double underscore separating MASS and KG, and the underscore folowing KG

df.rename(columns= {'PAYLOAD_MASS__KG_' : 'Payload_Mass_KG'}, inplace= True)

# For consistency sake and readability, I converted all of the column names to lowercase
# Note: in SQL, the column names are case insensitive, so it doesn't technically matter if the column names are upper, lower, or title case
df.columns = df.columns.astype(str).str.lower()

In [8]:
# I created a list of the column names so I could iterate over them below
cols = df.columns

# I created a loop to iterate over the different columns within the dataframe so I can quickly
# remove the extra white spaces within the column values
for x in cols:
    if df[x].dtype == 'object':
        df[x] = df[x].str.strip()

In [9]:
df.head()

,date,time (utc),booster_version,launch_site,payload,payload_mass_kg,orbit,customer,mission_outcome,landing_outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


---

In [10]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [11]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [12]:
%sql SELECT DISTINCT launch_site FROM spacextable;

 * sqlite:///my_data1.db
Done.


launch_site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [13]:
%sql SELECT launch_site, COUNT(*) AS 'Launch Count' FROM spacextable GROUP BY launch_site;

 * sqlite:///my_data1.db
Done.


launch_site,Launch Count
CCAFS LC-40,26
CCAFS SLC-40,34
KSC LC-39A,25
VAFB SLC-4E,16



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [14]:
%sql SELECT * FROM spacextable WHERE launch_site LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


date,time (utc),booster_version,launch_site,payload,payload_mass_kg,orbit,customer,mission_outcome,landing_outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [15]:
%sql SELECT customer, SUM(payload_mass_kg) AS 'Sum of Payload Mass (KG)' FROM spacextable WHERE customer = "NASA (CRS)";

 * sqlite:///my_data1.db
Done.


customer,Sum of Payload Mass (KG)
NASA (CRS),45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [16]:
%%sql 
SELECT Booster_Version, AVG(payload_mass_kg) AS 'Average Payload Mass (KG)' 
FROM spacextable WHERE Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


booster_version,Average Payload Mass (KG)
F9 v1.1,2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [17]:
%sql SELECT DISTINCT landing_outcome FROM spacextable;

 * sqlite:///my_data1.db
Done.


landing_outcome
Failure (parachute)
No attempt
Uncontrolled (ocean)
Controlled (ocean)
Failure (drone ship)
Precluded (drone ship)
Success (ground pad)
Success (drone ship)
Success
Failure


In [18]:
%%sql 
SELECT MIN(date) AS "First Successful Ground Pad Landing" 
FROM spacextable WHERE landing_outcome = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


First Successful Ground Pad Landing
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [19]:
%%sql 
SELECT booster_version, payload_mass_kg, landing_outcome 
FROM spacextable 
WHERE landing_outcome = 'Success (drone ship)' 
AND payload_mass_kg > 4000 AND payload_mass_kg < 6000;

 * sqlite:///my_data1.db
Done.


booster_version,payload_mass_kg,landing_outcome
F9 FT B1022,4696,Success (drone ship)
F9 FT B1026,4600,Success (drone ship)
F9 FT B1021.2,5300,Success (drone ship)
F9 FT B1031.2,5200,Success (drone ship)


### Task 7




##### List the total number of successful and failure mission outcomes


In [20]:
%sql SELECT mission_outcome, COUNT(*) AS 'Outcome Totals' FROM spacextable GROUP BY mission_outcome;

 * sqlite:///my_data1.db
Done.


mission_outcome,Outcome Totals
Failure (in flight),1
Success,99
Success (payload status unclear),1


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [21]:
%%sql 
SELECT DISTINCT booster_version AS 'Booster Version', payload_mass_kg AS 'Max Payload Mass (KG)'
FROM spacextable WHERE payload_mass_kg = (SELECT MAX(payload_mass_kg) FROM spacextable)
ORDER BY booster_version;

 * sqlite:///my_data1.db
Done.


Booster Version,Max Payload Mass (KG)
F9 B5 B1048.4,15600
F9 B5 B1048.5,15600
F9 B5 B1049.4,15600
F9 B5 B1049.5,15600
F9 B5 B1049.7,15600
F9 B5 B1051.3,15600
F9 B5 B1051.4,15600
F9 B5 B1051.6,15600
F9 B5 B1056.4,15600
F9 B5 B1058.3,15600


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [22]:
%%sql
SELECT CASE
    WHEN substr(date, 6, 2) = '01' THEN 'January'
    WHEN substr(date, 6, 2) = '02' THEN 'February'
    WHEN substr(date, 6, 2) = '03' THEN 'March'
    WHEN substr(date, 6, 2) = '04' THEN 'April'
    WHEN substr(date, 6, 2) = '05' THEN 'May'
    WHEN substr(date, 6, 2) = '06' THEN 'June'
    WHEN substr(date, 6, 2) = '07' THEN 'July'
    WHEN substr(date, 6, 2) = '08' THEN 'August'
    WHEN substr(date, 6, 2) = '09' THEN 'September'
    WHEN substr(date, 6, 2) = '10' THEN 'October'
    WHEN substr(date, 6, 2) = '11' THEN 'November'
    WHEN substr(date, 6, 2) = '12' THEN 'December'
END AS 'Month', 
landing_outcome, booster_version, launch_site
FROM spacextable
WHERE landing_outcome = 'Failure (drone ship)' AND substr(date, 0, 5) = '2015';

 * sqlite:///my_data1.db
Done.


Month,landing_outcome,booster_version,launch_site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


I used a `CASE` statement to satisfy the "display the month names" requirement. 
For numbers 1-9, I used the `LIKE` operator as a safeguard against formatting inconsistencies in the month. 

### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [23]:
%%sql
SELECT landing_outcome, COUNT(*) AS 'Landing Count' FROM spacextable
WHERE date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY landing_outcome
ORDER BY COUNT(*) DESC;

 * sqlite:///my_data1.db
Done.


landing_outcome,Landing Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
